In [1]:
import os
import sys
import random
import numpy as np
import scipy.io as scio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn import preprocessing
from sklearn.preprocessing import scale
import tensorflow as tf
from tensorflow import keras

In [3]:
N_train = 160
N_test = 36
image_sizeX = 128
image_sizeY = 128
num_channel = 1
train_data = []
train_mask1 = []
train_mask2 = []
test_data = []
test_mask1 = []
test_mask2 = []

In [4]:
# Load data
for i in range(N_train - 1):
    x = mpimg.imread('./dataset/data/%d.png'%(i+1))
    x = x.reshape(image_sizeX,image_sizeY,1)
    train_data.append(x)
    m1 = mpimg.imread('./dataset/mask1/%d.png'%(i+1))
    m1 = m1.reshape(image_sizeX,image_sizeY,1)
    train_mask1.append(m1)
    m2 = mpimg.imread('./dataset/mask2/%d.png'%(i+1))
    m2 = m2.reshape(image_sizeX,image_sizeY,1)
    train_mask2.append(m2)

for i in range(N_train, N_train+N_test):
    x = mpimg.imread('./dataset/data/%d.png'%(i+1))
    x = x.reshape(image_sizeX,image_sizeY,1)
    test_data.append(x)
    m1 = mpimg.imread('./dataset/mask1/%d.png'%(i+1))
    m1 = m1.reshape(image_sizeX,image_sizeY,1)
    test_mask1.append(m1)
    m2 = mpimg.imread('./dataset/mask2/%d.png'%(i+1))
    m2 = m2.reshape(image_sizeX,image_sizeY,1)
    test_mask2.append(m2)

In [5]:
train_data = np.array(train_data)
train_mask1 = np.array(train_mask1)
train_mask2 = np.array(train_mask2)
test_data = np.array(test_data)
test_mask1 = np.array(test_mask1)
test_mask2 = np.array(test_mask2)

In [6]:
def multi_scale(x, filters, padding="same", strides=1):
    c1 = keras.layers.Conv2D(filters, (1,1), padding=padding, strides=strides, activation="relu")(x)
    c3 = keras.layers.Conv2D(filters, (3,3), padding=padding, strides=strides, activation="relu")(x)
    c5 = keras.layers.Conv2D(filters, (3,3), padding=padding, strides=strides, activation="relu")(x)
    c7 = keras.layers.Conv2D(filters, (3,3), padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Concatenate()([c1, c3, c5, c7])
    return c

def down_block2(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    f = int(filters/4)
    c = multi_scale(x, f)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block2(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    f = int(filters/4)
    us = keras.layers.UpSampling2D((2, 2))(x)
    us = keras.layers.Conv2D(filters, (2, 2), padding='same', strides=1, activation="relu")(us)
    c = keras.layers.Concatenate()([us, skip])
    c = multi_scale(c, f)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck2(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    f = int(filters/4)
    c = multi_scale(x, f)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [7]:
def DMRF_UNet():
    f0 = 64
    f = [f0, f0*2]
    inputs = keras.layers.Input((image_sizeX, image_sizeY, 1))
    

    #### model_2
    pp0 = inputs
    cc1, pp1 = down_block2(pp0, f[0])
    
    bnn = bottleneck2(pp1, f[1])
    
    uu1 = up_block2(bnn, cc1, f[1])

    output_2 = keras.layers.Conv2D(1, (1, 1), padding="same", activation="elu")(uu1)
    model_2 = tf.keras.Model(inputs, output_2)

    return model_2

# Training
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

model = DMRF_UNet()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 16  32          ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 128, 128, 16  160         ['input_1[0][0]']                
                                )                                                             

In [8]:
total_epoch = 100
batch_size = 100
alpha = 10
beta = 1
path = '/'
model_path = path + 'exp/model.h5'
Adam = keras.optimizers.Adam(lr=1e-4)
lr_metric = get_lr_metric(Adam)
model.compile(optimizer=Adam, loss=['mse', 'mse'], loss_weights=[alpha, beta], metrics=['mae'])
model_checkpoint = keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True)
lr_checkpoint = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=2, min_lr=0)
history = model.fit(x=train_data, y=[train_mask1, train_mask2], batch_size=batch_size, epochs=total_epoch, verbose=2, \
    validation_data=(test_data, [test_mask1, test_mask2]), callbacks=[model_checkpoint])

Epoch 1/100


C:\Users\user\anaconda3\envs\python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)



Epoch 1: val_loss improved from inf to 2.07173, saving model to /exp\model.h5
2/2 - 65s - loss: 3.0284 - mae: 0.5470 - val_loss: 2.0717 - val_mae: 0.4523 - 65s/epoch - 32s/step
Epoch 2/100

Epoch 2: val_loss improved from 2.07173 to 1.29402, saving model to /exp\model.h5
2/2 - 47s - loss: 1.9060 - mae: 0.4332 - val_loss: 1.2940 - val_mae: 0.3560 - 47s/epoch - 24s/step
Epoch 3/100

Epoch 3: val_loss improved from 1.29402 to 0.64603, saving model to /exp\model.h5
2/2 - 55s - loss: 1.1554 - mae: 0.3349 - val_loss: 0.6460 - val_mae: 0.2482 - 55s/epoch - 28s/step
Epoch 4/100

Epoch 4: val_loss improved from 0.64603 to 0.18684, saving model to /exp\model.h5
2/2 - 52s - loss: 0.5415 - mae: 0.2242 - val_loss: 0.1868 - val_mae: 0.1229 - 52s/epoch - 26s/step
Epoch 5/100

Epoch 5: val_loss improved from 0.18684 to 0.07297, saving model to /exp\model.h5
2/2 - 38s - loss: 0.1379 - mae: 0.0961 - val_loss: 0.0730 - val_mae: 0.0634 - 38s/epoch - 19s/step
Epoch 6/100

Epoch 6: val_loss did not improve

KeyboardInterrupt: 

In [10]:
len(train_data)

159

In [11]:
model.save_weights('model_weights.h5')

In [26]:
model.load_weights(model_path)
model.evaluate(x=test_data, y=[test_mask1, test_mask2], batch_size=batch_size)
test_pred = model.predict(test_data)
for i in range(len(test_data)):
    pred1 = test_pred.reshape(768,768) * 255
    pred1 = Image.fromarray(pred1)
    pred1.convert('L').save('./exp/visual/1pred%d.png'%(i+1))

2/2 [==============================] - 1s 151ms/step


In [19]:
print(np.size(test_pred))

589824
